## Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv("datos/df_clean.csv", index_col=0)

In [6]:
df.columns

Index(['superior_body_code', 'name_superior_body', 'organ_code', 'organ_name',
       'management_unit_code', 'name_management_unit', 'economy_category',
       'source_revenue', 'recipe_species', 'detailing',
       'updated_estimated_value', 'value_launched', 'realized_value',
       'percentage_realized', 'release_date', 'financial_year',
       'economy_subcategory'],
      dtype='object')

Vamos a crear un nuevo dataframe que contenga los ingresos por cateogoría económica.

In [32]:
df_value_per_category = df.groupby(["economy_category","economy_subcategory"])[["realized_value"]].sum()
df_value_per_category["percentage of total"] = (df_value_per_category["realized_value"]/df["realized_value"].dropna().sum()).round(4)

df_value_per_category

realized_value  percentage of total
economy_category    economy_subcategory                                     
Receitas Correntes  extra-orçamentárias    1.341424e+13               0.5073
                    intra-orçamentárias    2.808236e+11               0.0106
Receitas de Capital extra-orçamentárias    1.237381e+13               0.4680
                    intra-orçamentárias    2.142953e+10               0.0008
Sem informação      sem informação         3.498643e+11               0.0132

Vemos que ambas categorías principales (corrientes y de capital), reportan una proporción de ingresos totales comparable, aunque ligeramente superior en los ingresos corrientes, y que predominan aquellos que no son inter gubernamentales.

Ahora vamos a calcular la diferencia promedio entre ingresos previstos y realizados por categoría. Para ello crearemos una nueva columna en nuestro dataframe que contenga las diferencias entre estos datos.

In [37]:
df["difference_est_realized"] = (df["updated_estimated_value"]-df["realized_value"])
df.groupby(["economy_category", "economy_subcategory"])[["difference_est_realized"]].mean()

difference_est_realized
economy_category    economy_subcategory                         
Receitas Correntes  extra-orçamentárias            -4.778544e+05
                    intra-orçamentárias            -2.538572e+05
Receitas de Capital extra-orçamentárias             5.451779e+07
                    intra-orçamentárias            -1.106782e+08
Sem informação      sem informação                  3.935506e+07

Sin embargo, en secciones previas (limpieza de datos) podíamos observar muchos casos en los que el valor estimado era 0, por lo que el hecho de que tantos valores sean negativos se dan por este hecho. Vamos a calcular los casos donde el valor estimado no existe pero el realizado sí.

In [44]:
no_est_yes_realized = (df["updated_estimated_value"].isna() | df["updated_estimated_value"] == 0) & (df["realized_value"].notna() & df["realized_value"] != 0)
df_noest = df[no_est_yes_realized]
f'Las condiciones se cumplen en el {df_noest.shape[0]/df.shape[0]*100:.2f}% de los casos totales'

'Las condiciones se cumplen en el 92.46% de los casos totales'

Vemos que es un caso muy común. Esto puede ser porque estos ingresos no estuvieron planificados al inicio, y finalmente reportaron ingresos, o simplemente un error de registro. Si cogemos estos datos vemos que de hecho las medias de las diferencias disminuyen (más negativas) en todos los casos, indicando que es la principal fuente de estos datos tan negativos.

In [45]:
df_noest["difference_est_realized"] = (df_noest["updated_estimated_value"]-df_noest["realized_value"])
df_noest.groupby(["economy_category", "economy_subcategory"])[["difference_est_realized"]].mean()

C:\Users\yanru\AppData\Local\Temp\ipykernel_31548\1511455330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noest["difference_est_realized"] = (df_noest["updated_estimated_value"]-df_noest["realized_value"])


difference_est_realized
economy_category    economy_subcategory                         
Receitas Correntes  extra-orçamentárias            -1.121207e+07
                    intra-orçamentárias            -1.315709e+07
Receitas de Capital extra-orçamentárias            -3.530432e+08
                    intra-orçamentárias            -2.451876e+08
Sem informação      sem informação                 -1.620202e+07

Aclarar que cuanto más negativo, más retorno respecto a lo estimado.

Podemos realizar un análisis en los datos sin estos casos, el cual podría arrojar un poco más de luz en el análisis de las categorías, aunque hay que tener en cuenta que esta muestra representa menos de un 8% de los casos totales.

In [49]:
df_con_est = df[~no_est_yes_realized]
df_con_est["difference_est_realized"] = (df_con_est["updated_estimated_value"]-df_con_est["realized_value"])
df_con_est.groupby(["economy_category", "economy_subcategory"])[["difference_est_realized"]].mean()

C:\Users\yanru\AppData\Local\Temp\ipykernel_31548\3129268069.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_con_est["difference_est_realized"] = (df_con_est["updated_estimated_value"]-df_con_est["realized_value"])


difference_est_realized
economy_category    economy_subcategory                         
Receitas Correntes  extra-orçamentárias             4.256730e+08
                    intra-orçamentárias             2.198954e+08
Receitas de Capital extra-orçamentárias             7.437965e+09
                    intra-orçamentárias             5.404206e+09
Sem informação      sem informação                  1.341034e+09

Ahora los datos son todos positivos, lo cual indicaría que se ha recaudado en todos los casos una cantidad menor a la estimada inicialmente. Sin embargo, al igual que antes, registros donde sí que había una estimación pero no un valor realizado podrían estar detrás de estos valores tan grandes que obtenemos. Estos casos se podrían interpretar como registros donde se estimó una cantidad inicial que al final no recaudaron nada o simplemente, una vez más, un fallo en los registros. Una explicación más podría ser que este registro indica un ingreso aún no recibido, por lo que en el campo de valor realizado aparecería un vacío.

In [54]:
df_con_est_realized = df_con_est[(df_con_est["realized_value"] != 0) & (df_con_est["realized_value"].notna())]
df_con_est_realized["difference_est_realized"] = (df_con_est_realized["updated_estimated_value"]-df_con_est_realized["realized_value"])
df_con_est_realized.groupby(["economy_category", "economy_subcategory"])[["difference_est_realized"]].mean()

C:\Users\yanru\AppData\Local\Temp\ipykernel_31548\3219727869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_con_est_realized["difference_est_realized"] = (df_con_est_realized["updated_estimated_value"]-df_con_est_realized["realized_value"])


difference_est_realized
economy_category    economy_subcategory                         
Receitas Correntes  extra-orçamentárias             4.505536e+08
                    intra-orçamentárias             1.193413e+08
Receitas de Capital extra-orçamentárias             2.129793e+10
                    intra-orçamentárias                      NaN
Sem informação      sem informação                  4.687883e+09

Vemos ahora que si cogemos solo datos con ambos registros de valores obtenemos algo bastante similar al caso anterior (casi todos en el mismo orden de magnitud). Esto puede indicar que los casos sin valor realizado no hayan supuesto un gran cambio con el caso anterior.

Como conclusión de este apartado, sería crucial investigar el origen de los valores 0 o nulos de las columnas de valor estimado y realizado, así como su significado en nuestros datos. Esto nos permitiría decidir si corresponde que se eliminen o se haga un análisis separado con éstos. 

Con el análisis realizado podríamos deducir que, sin descartar ningún registro, el gobierno brasileño consigue batir sus estimaciones en ingresos corrientes, tanto extra como inter gubernamentales, mientras que en los ingresos de capital ingresa menos de lo estimado, específicamente en los movimientos extra gubernamentales. Este suceso no ocurre en aquellos de interior.

Eliminando los registros sin estimación nos otorga unos resultados bastante diferentes, ya que en este caso ninguna categoría económica es capaz de ingresar más de lo estimado. De hecho, basado en estos datos, de media el gobierno tiene unos ingresos bastante más bajos a lo que estiman, a lo largo de todas las categorías. Cabe indicar que, como ha sido mencionado previamente, estos datos podrían no ser representativos de la realidad, ya que no tenemos información suficiente sobre los vacíos ni los 0 en ninguno de los valores, tanto estimados como realizados.